<a href="https://colab.research.google.com/github/madhulikag/BanaNET/blob/main/Bananutrient_multi_class_EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import libraries
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2, MobileNetV3Large
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, Conv2D, DepthwiseConv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
from PIL import Image


import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Image directory, change to yours as needed
data_dir = '/content/drive/MyDrive/Banana_leaf'

image_size = (224, 224)
batch_size = 32


In [5]:
# Data Processing for unorganized directory
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode = 'categorical',
    subset = 'training'
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode = 'categorical',
    subset = 'validation',
    shuffle = False
)

Found 2657 images belonging to 4 classes.
Found 664 images belonging to 4 classes.


In [6]:
# Data Processing for organized directory
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Banana_leaf_split/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Banana_leaf_split/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle = False
)

Found 2657 images belonging to 4 classes.
Found 664 images belonging to 4 classes.


In [7]:
# input shape: image size and RGB
base_model = MobileNetV3Large(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze base model weights for now
base_model.trainable = False

inputs = tf.keras.Input(shape=(224,224,3))
x = base_model(inputs, training=False)  # pass inputs through base model
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,        # Reduce LR by 50%
    patience=2,        # Wait 2 epochs before reducing
    min_lr=1e-6,       # Don't go below this LR
    verbose=1
)



In [ ]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=[reduce_lr, early_stop, checkpoint]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
43/84 ━━━━━━━━━━━━━━━━━━━━ 4:31 7s/step - accuracy: 0.2467 - loss: 1.8713

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Predict probabilities
y_pred_probs = model.predict(val_generator)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)

# True labels from the generator
y_true = val_generator.classes

In [ ]:
from sklearn.metrics import classification_report
# Generate classification report with class names
class_labels = list(val_generator.class_indices.keys())

print(classification_report(y_true, y_pred, target_names=class_labels))

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)

# Generate Confusion matrix with class names
class_labels = list(val_generator.class_indices.keys())

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels,
            yticklabels=class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Grab the MobileNetV3 submodel (by name)
base_model = model.get_layer("MobileNetV3Large")

# Find its last convolutional layer
for layer in reversed(base_model.layers):
    if isinstance(layer, (Conv2D, DepthwiseConv2D)):
        last_conv_layer_name = layer.name
        break
print("Using conv layer:", last_conv_layer_name)

In [ ]:
# Pick an example image from your validation set
#    (or load one from disk)
img_path = val_generator.filepaths[0]      # first val image
orig = cv2.imread(img_path)
orig = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
resized = cv2.resize(orig, image_size)

# Preprocess & batch it
x = preprocess_input(resized.astype(np.float32))
x = np.expand_dims(x, axis=0)  # shape = (1,224,224,3)

In [ ]:
# Run Grad-CAM
heatmap = make_gradcam_heatmap(x, model, last_conv_layer_name, base_model)

# Overlay the heatmap on the original image
heatmap = cv2.resize(heatmap, (orig.shape[1], orig.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

overlay = cv2.addWeighted(orig, 0.6, heatmap, 0.4, 0)

# Show results
plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
plt.title("Original")
plt.imshow(orig); plt.axis("off")
plt.subplot(1,3,2)
plt.title("Heatmap")
plt.imshow(heatmap); plt.axis("off")
plt.subplot(1,3,3)
plt.title("Overlay")
plt.imshow(overlay); plt.axis("off")
plt.show()

In [ ]:
model.summary()

In [ ]:
base_model = model.layers[0]
for layer in base_model.layers[::-1]:  # reverse order
    if 'conv' in layer.name:
        print(layer.name)
        break  # first one found in reverse is the last conv layer

In [ ]:
last_conv_name = "top_conv"

In [ ]:
from tensorflow.keras.preprocessing import image
# Path to the test image (adjust as needed)
img_path = '/content/drive/MyDrive/Banana_leaf/potassium/k3.jpg'

img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, axis=0)
img_array = tf.keras.applications.mobilenet_v3.preprocess_input(img_array)

In [ ]:
heatmap = make_gradcam_heatmap(img_array, model, last_conv_name, base_model)

In [ ]:
data_dir = '/content/drive/MyDrive/Banana_leaf'


image_size = (300, 300)
batch_size = 32

In [ ]:
# Data Processing for unorganized directory
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode = 'categorical',
    subset = 'training'
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode = 'categorical',
    subset = 'validation',
    shuffle = False
)

In [ ]:
base_model = EfficientNetB3(
    include_top=False,
    weights='imagenet',
    input_shape=(300, 300, 3)  # EfficientNetB3 expects 300x300
)
base_model.trainable = False  # Freeze initially

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,        # Reduce LR by 50%
    patience=2,        # Wait 2 epochs before reducing
    min_lr=1e-6,       # Don't go below this LR
    verbose=1
)

In [ ]:
model = Sequential([
    Input(shape=(300, 300, 3)),
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')  # change to match your number of classes
])
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=[reduce_lr, early_stop, checkpoint]
)

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Predict probabilities
y_pred_probs = model.predict(val_generator)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)

# True labels from the generator
y_true = val_generator.classes

In [ ]:
# Generate classification report with class names
class_labels = list(val_generator.class_indices.keys())

print(classification_report(y_true, y_pred, target_names=class_labels))

In [ ]:
cm = confusion_matrix(y_true, y_pred)

# Generate Confusion matrix with class names
class_labels = list(val_generator.class_indices.keys())

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels,
            yticklabels=class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()